In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import scipy.stats as sps

**1. Батч-эффект, связанный с TSS.**
- Распределение 0/1/2 (одномодально, двумодально) по TSS и разным ракам (2 - бимодально; 1 - выше 11; 0 - ниже 11)
- Есть ли бимодальность вне TCGA? Найти miRNA-seq выборки пациентов какого-то рака.

In [155]:
bimodal = ['BRCA', 'COAD', 'HNSC', 'LUAD', 'LUSC', 'READ', 'UCEC']

In [156]:
def get_tss(name_cancer):
    df = pd.read_csv(name_cancer+'_isomiR_5prime_CPM.tsv', sep = '\t')
    mir22 = df.loc["hsa-miR-22-3p|+1"]
    
    cond_cancer = df.columns.str.contains('-0[1-9][A-Z]', regex = True)
    df_cancer = df[df.columns[cond_cancer]]
    
    df_mir22 = df_cancer.loc['hsa-miR-22-3p|+1']
    df_mir22.index = list(map(lambda i: i[5:7], df_mir22.index))
    
    dic_TSS = {}.fromkeys(list(set(df_mir22.index)), '')
    for i in dic_TSS:
        more, less = np.quantile(df_mir22.loc[i], 0.75) > 11, np.quantile(df_mir22.loc[i], 0.25) < 11
        dic_TSS[i] = [2 if (more and less) else 0 if (less and not more) else 1][0]
    return dic_TSS

In [157]:
dict_tss_cancer = {}.fromkeys(bimodal, {})
set_tss = set()
for i in bimodal:
    dict_tss_cancer[i] = get_tss(i)
    set_tss |= set(get_tss(i).keys())
set_tss = list(set_tss)

In [158]:
# for i, j in dict_tss_cancer.items():
#     print(i)
#     for k, l in j.items():
#         if l == 2:
#             print(k, end = ', ')
#     print(' ')

In [159]:
data = pd.DataFrame(np.empty((len(bimodal), len(set_tss))),
            index = bimodal,
            columns = set_tss).T
data[:] = np.nan

In [160]:
for cancer in data.columns:
    t = dict_tss_cancer[cancer]
    for i, j in t.items():
        data.loc[i, cancer] = j

In [161]:
data

,BRCA,COAD,HNSC,LUAD,LUSC,READ,UCEC
WA,NaN,NaN,0.0,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,0.0,NaN,NaN
73,NaN,NaN,NaN,2.0,NaN,NaN,NaN
WT,0.0,NaN,NaN,NaN,NaN,NaN,NaN
CA,NaN,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
L9,NaN,NaN,NaN,0.0,NaN,NaN,NaN
SL,NaN,NaN,NaN,NaN,NaN,NaN,0.0
51,NaN,NaN,NaN,NaN,1.0,NaN,NaN
DQ,NaN,NaN,0.0,NaN,NaN,NaN,NaN
